# 6. Purely functional state

## 6.1 Generating random numbers using side effects

In [68]:
val rng=new scala.util.Random


scala.util.Random@c64486

In [69]:
rng.nextDouble

0.4309352267947695

In [70]:
rng.nextDouble

0.4815440601951334

In [71]:
rng.nextInt

1864556156

In [72]:
def rollingDie: Int = {
    val rng = new scala.util.Random
    rng.nextInt(6)
}

In [73]:
rollingDie

0

In [74]:
rollingDie

5

In [75]:
rollingDie

0

=> all using internal state variable => SIDE EFFECT!

## 6.2 Purely functional random number generation

In [76]:
trait RNG {
    def nextInt: (Int, RNG)
}

### => 난수 계산과 state 전달 분리.

In [77]:
case class SimpleRNG(seed: Long) extends RNG {
    def nextInt: (Int, RNG) = {
        val newSeed = (seed * 0x5DEECE66DL + 0xBL) & 0xFFFFFFFFFFFFL
        val nextRNG = SimpleRNG(newSeed)
        val n = (newSeed >>> 16).toInt
        (n, nextRNG)
    }
}

In [78]:
val rng = SimpleRNG(42)

SimpleRNG(42)

In [79]:
val (n1, rng2) = rng.nextInt

16159453

In [80]:
val (n2, rng3) = rng2.nextInt

-1281479697

In [81]:
rng3

SimpleRNG(197491923327988)

In [82]:
rng2

SimpleRNG(1059025964525)

In [83]:
rng

SimpleRNG(42)

## 6.3 Making stateful APIs pure

In [84]:
def randomPair0(rng:RNG):(Int, Int) = {
    val (i1, _) = rng.nextInt
    val (i2, _) = rng.nextInt
    (i1,i2)
}

In [85]:
randomPair0(rng2)

(-1281479697,-1281479697)

In [86]:
def randomPair(rng:RNG): ((Int,Int), RNG) = {
    val (i1, rng2) = rng.nextInt
    val (i2, rng3) = rng2.nextInt
    ((i1,i2),rng3)
}

In [87]:
randomPair(rng2)

((-1281479697,-340305902),SimpleRNG(259172689157871))

### Ex.6.1 nonNegativeInt [0 ~ Int.maxValue]

In [88]:
def nonNegativeInt(rng:RNG): (Int, RNG) = {
    var n = 0
    val (i, rngNext) = rng.nextInt
    if (i==Int.MinValue) n = - (Int.MinValue+1)
    if (i<0) n = -i
    else n = i
    (n,rngNext)
// book's answer
// if (i < 0) (-(n+1),rngNext) else (i, rngNext) 
}

In [89]:
val (n1, r3) = nonNegativeInt(rng2)

1281479697

In [90]:
r3

SimpleRNG(197491923327988)

In [91]:
nonNegativeInt(r3)

(340305902,SimpleRNG(259172689157871))

> As you write more functional programs, you’ll sometimes encounter situations like
this where the functional way of expressing a program feels awkward or tedious. Does
this imply that purity is the equivalent of trying to write an entire novel without using
the letter E? Of course not. ** Awkwardness like this is almost always a sign of some
missing abstraction waiting to be discovered.**

### Ex.6.2 double

In [92]:
def double(rng:RNG): (Double, RNG) = {
    val (i,r) = nonNegativeInt(rng)
    if (i!=Int.MaxValue) ((i.toDouble / Int.MaxValue.toDouble), r)
    else ((Int.MaxValue-1).toDouble / Int.MaxValue.toDouble, r)
}

In [93]:
double(rng2)

(0.5967354856416283,SimpleRNG(197491923327988))

In [94]:
def intDouble(rng:RNG): ((Int,Double), RNG) = {
    val (i, r1) = rng.nextInt
    val (d, r2) = double(r1)
    ((i,d), r2)
}

In [95]:
intDouble(rng2)

((-1281479697,0.15846728447753344),SimpleRNG(259172689157871))

### Ex.6.3 doubleInt, intDouble, double3

In [96]:
def doubleInt(rng:RNG): ((Double,Int), RNG) = {
    val (i, r1) = rng.nextInt
    val (d, r2) = double(r1)
    ((d, i), r2)
}

In [97]:
doubleInt(rng2)

((0.15846728447753344,-1281479697),SimpleRNG(259172689157871))

In [98]:
def double3(rng:RNG): ((Double,Double,Double),RNG) = {
    val (d1, r1) = double(rng)
    val (d2, r2) = double(r1)
    val (d3, r3) = double(r2)
    ((d1,d2,d3),r3)
}

In [99]:
double3(rng2)

((0.5967354856416283,0.15846728447753344,0.9386595436086224),SimpleRNG(149370390209998))

### Ex.6.4 ints

In [115]:
def ints(count:Int)(rng:RNG): (List[Int], RNG) = {
    def go(n:Int,rl:List[Int],rngIn:RNG): (List[Int],RNG) = {
        if (n==0) (rl, rngIn)
        else {
            val (i,rngNext) = rngIn.nextInt
            go(n-1, rl :+ i, rngNext)
        }
    }
    go(count,List(),rng)
}

In [116]:
ints(5)(rng2)

(List(-1281479697, -340305902, -2015756020, 1770001318, -1934589059),SimpleRNG(154689748186168))

## 6.4 A better API for state actions

### common pattern("state actions", "state transitions"): 
        RNG => (A, RNG)

### RNG state action data type:

In [117]:
type Rand[+A] = RNG => (A, RNG)

#### nextInt as a type of Rand[Int]

In [118]:
val int: Rand[Int] = _.nextInt

<function1>

### combinators - unit (do nothing)

In [119]:
def unit[A](a:A): Rand[A] = 
    rng => (a,rng)

### combinators - map (function composition)

In [120]:
def map[A,B](s:Rand[A])(f: A => B): Rand[B] = 
    rng => {
        val (a, rng2) = s(rng)
        (f(a), rng2)
    }

#### using map to implement nonNegativeEven

In [121]:
def nonNegativeEven: Rand[Int] =
    map(nonNegativeInt)(i => i - i % 2)

In [122]:
nonNegativeEven(rng2)

(1281479696,SimpleRNG(197491923327988))

In [123]:
nonNegativeEven(rng3)

(340305902,SimpleRNG(259172689157871))

### Ex.6.5 double(using 'map'. cf. Ex.6.2)

In [124]:
def convToDouble(i:Int): Double = {
    val maxFloat = Int.MaxValue.toDouble
    if (i!=Int.MaxValue) i.toDouble / maxFloat
    else (i-1).toDouble / maxFloat
}
    
def doubleViaMap: Rand[Double] = 
    map(nonNegativeInt)(i => convToDouble(i))

In [125]:
doubleViaMap(rng2)

(0.5967354856416283,SimpleRNG(197491923327988))

In [126]:
double(rng2)

(0.5967354856416283,SimpleRNG(197491923327988))